<a href="https://colab.research.google.com/github/weathon/kNN_Research/blob/main/Credit_Card_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cp kaggle.json /root/.kaggle/
!kaggle datasets download -d sakshigoyal7/credit-card-customers
!unzip credit-card-customers.zip

credit-card-customers.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  credit-card-customers.zip
  inflating: BankChurners.csv        


In [ ]:
import pandas as pd
import numpy as np

### Data Washing

In [196]:
BankChurners = pd.read_csv("BankChurners.csv")

In [197]:
del BankChurners["Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1"]
del BankChurners["Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"]

In [198]:
Y = BankChurners["Attrition_Flag"]
del BankChurners["Attrition_Flag"]
del BankChurners["CLIENTNUM"]
X = BankChurners

In [199]:
Y = Y.replace("Existing Customer",-1).replace("Attrited Customer",1)

In [200]:
X["Gender"] = X["Gender"].replace("M",1).replace("F",0)
X["Marital_Status"] = X["Marital_Status"].replace("Married",0).replace("Single",1).replace("Divorced",2).replace("Unknown",3)
X["Education_Level"] = X["Education_Level"].replace("Uneducated",0).replace("High School",1).replace("College",2).replace("Graduate",3).replace("Post-Graduate",4).replace("Doctorate",5).replace("Unknown",-1)
X["Card_Category"] = X["Card_Category"].replace("Blue",0).replace("Silver",1).replace("Gold",2).replace("Platinum",3)
X["Income_Category"] = X["Income_Category"].replace("Less than $40K",0).replace("$40K - $60K",1).replace("$60K - $80K",2).replace("$80K - $120K",3).replace("$120K +",4).replace("Unknown",-1)

### Baseline: tradional kNN

In [256]:
def result(x):
  return 1 if x>0 else -1

In [331]:
X = np.array(X)
Y = np.array(Y)
indexs = np.arange(len(X))
np.random.shuffle(indexs)
X = X[indexs]
Y = Y[indexs]

r = int(len(X)*0.9)
npX = np.array(X[:r])
npY = np.array(Y[:r])
X_val = np.array(X[r:])
Y_val = np.array(Y[r:])

error = 0
TruePositives = 0
FalseNegatives = 0
FalsePositives = 0
TrueNegatives = 0
MSE = 0

for sample in range(len(X)-r): # change this from 20 to r decreased the accuracy a alot
  distance = np.sum((npX - X_val[sample])**2,axis=1)
  sum = 0
  for i in range(100):
    m = np.argmin(distance)
    distance[m] = 1000000000 
    sum += 1/(distance[m]+0.000000001) * (npY[m]) #ksougjuqkuamgerdoing

  if(Y_val[sample] != result(sum)):
    error+=1
  
  if Y_val[sample] == result(sum) == 1:
    TruePositives += 1

  if result(sum) == -1 and Y_val[sample] == 1:
    FalseNegatives +=1

  if result(sum) == 1 and Y_val[sample] == -1:
    FalsePositives +=1

  if Y_val[sample] == result(sum) == -1:
    TrueNegatives += 1


print(1-error/r)
print(TruePositives / (TruePositives + FalseNegatives))
print(TruePositives / (TruePositives + FalsePositives))
print(MSE/(len(X)-r)) #niao zenmjisuanprobility? ratio!!!!!!! why i am doing plus or minus aratio 

0.9849681808207154
0.3492063492063492
0.825
0.9999998708430405


In [328]:
distance[8]

4043056.074194